# ASIC Corporate Insolvency Data

Make sure you save the latest data in the cache dierctory and update cell 6, before running this notebook. 

## Python set-up

In [1]:
# system imports
from pathlib import Path
import textwrap

In [2]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# local imports
from plotting import line_plot, set_chart_dir, finalise_plot
from decompose import decompose

Could not import auto_arima from pmdarima


In [4]:
# plotting set-up
TITLE = "Corporate Insolvencies"
SOURCE = "Source: ASIC"
LFOOTER = "The first time a company enters external administration or has a controller appointed"
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/ASIC/"
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)
set_chart_dir(CHART_DIR)
for filename in Path(CHART_DIR).glob("*.png"):
    filename.unlink()
SHOW = False

## State/National Data Capture

In [5]:
ASIC_CACHE_DIR = "./ASIC_CACHE/"

In [6]:
# YOU MUST DOWNLOAD THE LATEST DATA FROM THE ASIC WEBSITE
# and save it in the ASIC_CACHE_DIR directory
# Then amend the filename in the next line oif code ...
LATEST_DATA = (
    ASIC_CACHE_DIR
    + "asic-insolvency-statistics-series-1-and-series-2-published-2-april-2024.xlsx"
)

# see: https://asic.gov.au/regulatory-resources/find-a-document/statistics/insolvency-statistics/insolvency-statistics-current/


def fix_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Fix the column names in the ASIC data."""

    rename = {x: x.title() for x in df.columns}
    df = df.rename(columns=rename)

    renamer = {x: x.replace("\n", "") for x in df.columns}
    df = df.rename(columns=renamer)

    renamer = {x: x.replace("&", " and ") for x in df.columns}
    df = df.rename(columns=renamer)

    renamer = {x: x.replace("And", "and") for x in df.columns}
    df = df.rename(columns=renamer)

    renamer = {x: x.replace("  ", " ") for x in df.columns}
    df = df.rename(columns=renamer)

    return df


def get_asic_data(
    sheet: str = "1.3",
    pm: str = "Period(Month)",
    pcy: str = "Period(Calendar Year)",
):
    """Get the latest ASIC data."""

    asic_data = pd.read_excel(LATEST_DATA, sheet_name=sheet, skiprows=10)
    asic_data = fix_columns(asic_data)
    asic_data = asic_data.dropna(how="all", axis=1)  # drop nan columns
    asic_data = asic_data.loc[asic_data[pm].notna()]  # drop total rows
    # get monthly PeriodIndex
    asic_data.index = pd.PeriodIndex(
        asic_data[pcy].astype(int).astype(str) + "-" + asic_data[pm],
        freq="M",
    )
    asic_data = asic_data.rename(columns={"Total": "Australia"})

    return asic_data


current_data = get_asic_data()

In [7]:
def get_asic_history() -> pd.DataFrame:
    """Get ASIC insolvency history data."""

    # get the monthly series-1 data
    asic_history_monthly = "1.2"
    asic_history = (
        ASIC_CACHE_DIR
        + "asic-insolvency-statistics-series-1-published-8-september-2022.xlsx"
    )
    asic_data = pd.read_excel(asic_history, sheet_name=asic_history_monthly, skiprows=4)
    asic_data = fix_columns(asic_data)

    # remove junk rows
    asic_data = asic_data.dropna(axis=0, how="all")
    asic_data = asic_data.iloc[1:-1]

    # fix the strange date format
    asic_data["Fin Year"] = asic_data.loc[asic_data["Australia"].isna(), "Period"]
    asic_data["Fin Year"] = asic_data["Fin Year"].ffill()
    asic_data = asic_data.dropna(thresh=8, axis=0)
    asic_data["Year"] = (
        asic_data["Fin Year"]
        .str.split("-")
        .str[0]
        .where(
            asic_data["Period"].isin(
                ("July", "August", "September", "October", "November", "December")
            ),
            other=asic_data["Fin Year"].str.split("-").str[1],
        )
    )
    asic_data.index = pd.PeriodIndex(
        asic_data["Year"] + "-" + asic_data["Period"], freq="M"
    )

    return asic_data


historic_data = get_asic_history()

## State/National plots

In [8]:
def combine_data[T: (pd.Series, pd.DataFrame)](history: T, current: T) -> T:
    """Combine historic and current data, which is either a Series or a DataFrame.
    Note: This typing definition requires Python 3.12"""
    combined_data = pd.concat([history, current], axis=0)
    combined_data = combined_data[~combined_data.index.duplicated(keep="last")]
    combined_data = combined_data.sort_index()
    return combined_data

In [9]:
def plot_trend():
    """Plot trend and seasonally adjusted series."""
    combined_series = combine_data(
        historic_data["Australia"], current_data["Australia"]
    )
    decomp = decompose(s=combined_series.iloc[:-1], model="multiplicative")

    line_plot(
        decomp[["Seasonally Adjusted", "Trend"]],
        starts=(None, "2019-01"),
        title=f"{TITLE}: Australia",
        width=(1, 2),
        ylabel="First-time Insolvencies/Month",
        rfooter=SOURCE,
        rheader="Locally calculated seasonal adjustment",
        lheader="Last month of (provisional) data excluded",
        lfooter=LFOOTER,
        show=SHOW,
    )


plot_trend()

In [10]:
def state_growth():
    """Calculate and plot state growth rates."""

    states = [
        "New South Wales",
        "Victoria",
        "Queensland",
        "South Australia",
        "Western Australia",
        "Tasmania",
        "Northern Territory",
        "Australian Capital Territory",
        "Australia",
    ]
    combined = combine_data(historic_data[states], current_data[states]).iloc[
        :-1
    ]  # ignore last row

    too_long = 58
    for col in combined:
        title = f"{TITLE}: {col} (Original series)"
        separator = "\n" if len(title) > too_long else " "
        title = title.replace(" (", f"{separator}(")
        line_plot(
            combined[col],
            title=title,
            ylabel="First-time Insolvencies/Month",
            width=1,
            rfooter=SOURCE,
            lheader="Last month of (provisional) data excluded",
            lfooter=LFOOTER,
            show=SHOW,
        )

    # calculate growth rates
    state_data = combined[states]
    latest = state_data.iloc[-12:].sum()
    base_start, base_end = 2015, 2019
    years = base_end - base_start + 1
    select = (state_data.index.year >= base_start) & (state_data.index.year <= base_end)
    base = state_data.loc[select].sum() / years
    growth = ((latest / base) - 1) * 100
    # print(f"{base=}\n{latest=}\n{growth=}")

    # plot growth rates
    _fig, ax = plt.subplots()
    growth.sort_values().plot(kind="barh", ax=ax)
    finalise_plot(
        ax,
        title=f"{TITLE} in the 12 months\nto {combined.index[-1]} by State over {base_start}-{base_end} Ave",
        xlabel="Growth (%)",
        rfooter=SOURCE,
        lheader="Last month of (provisional) data excluded",
        lfooter=LFOOTER,
        show=SHOW,
    )


state_growth()

## By Industry Sector

In [11]:
industrial = [
    "Accommodation and Food Services",
    "Administrative and Support Services",
    "Agriculture, Forestry and Fishing",
    "Arts and Recreation Services",
    "Construction",
    "Education and Training",
    "Electricity, Gas, Water and Waste Services",
    "Financial and Insurance Services",
    "Health Care and Social Assistance",
    "Information Media and Telecommunications",
    "Manufacturing",
    "Mining",
    "Other Services",
    "Professional, Scientific and Technical Services",
    "Public Administration and Safety",
    "Rental, Hiring and Real Estate Services",
    "Retail Trade",
    "Transport, Postal and Warehousing",
    "Wholesale Trade",
    "Unknown",
    "Australia",
]

In [12]:
def is_history() -> pd.DataFrame:
    """Get ASIC insolvency history data by industry."""

    file = (
        ASIC_CACHE_DIR
        + "asic-insolvency-statistics-series-1a-published-8-september-2022.xlsx"
    )
    asic_data = pd.read_excel(io=file, sheet_name="1A.1.2", skiprows=5).dropna(
        thresh=8, axis=0
    )
    asic_data = fix_columns(asic_data)
    asic_data = asic_data[asic_data["Period and Region"].str.contains("Total for")]

    # amalgamate FIS
    fis = [
        "Financial and Insurance Services",
        "Fis–Credit Provider",
        "Fis–Deposit Taking Institutions",
        "Fis–Insurance",
        "Fis–Managed Investments",
        "Fis–Other Financial Services",
        "Fis–Superannuation",
    ]
    fis_rows = asic_data[fis].sum(axis=1, skipna=True)
    asic_data = asic_data.drop(columns=fis)
    asic_data[fis[0]] = fis_rows
    cols = list(asic_data.columns)
    to_move = cols.pop(-1)
    cols.insert(9, to_move)
    asic_data = asic_data[cols]

    # rename columns to match with current data
    asic_data = asic_data.rename(
        columns={"Other (Business and Personal) Services": "Other Services"}
    )
    asic_data = asic_data.rename(
        columns={
            "Information Media and Tele- Communications": "Information Media and Telecommunications"
        }
    )
    asic_data = asic_data.rename(columns={"Total": "Australia"})

    # get monthly PeriodIndex
    asic_data["Period and Region"] = asic_data["Period and Region"].str.replace(
        "Total for ", ""
    )
    index = [
        pd.Period(x, freq="M")
        for x in
        # Note: some data quality issues being addressed here
        asic_data["Period and Region"]
        .str.replace("NOVMEBER", "NOVEMBER", case=False)  # data quality fix
        .str.title()
        .str.split(" ")[::-1]
        .str.join("-")
        .str.replace("--", "-")[::-1]  # data quality fix
    ]
    asic_data.index = index
    return asic_data


is_hist = is_history()[industrial]

In [13]:
def plot_industrial():
    """Plot industry-sector data."""

    is_current_data = get_asic_data(
        sheet="1.2", pm="Period (Month)", pcy="Period (Calendar Year)"
    )[industrial]
    is_current_data = is_current_data.iloc[:-1]  # drop last provisional value

    combined = combine_data(is_hist, is_current_data)
    for col in industrial:
        title = "\n".join(textwrap.wrap(f"{TITLE}: {col} (Original series)", 50))
        line_plot(
            combined[col],
            title=title,
            ylabel="First-time Insolvencies/Month",
            width=1,
            rfooter=SOURCE,
            lheader="Last month of (provisional) data excluded",
            lfooter=LFOOTER,
            show=SHOW,
        )

    # calculate growth rates
    latest = is_current_data.iloc[-12:].sum()
    base_start, base_end = 2015, 2019
    years = base_end - base_start + 1
    select = (combined.index.year >= base_start) & (combined.index.year <= base_end)
    base = combined.loc[select].sum() / years
    growth = ((latest / base) - 1) * 100
    cutoff = 150  # per year
    growth = growth[base > cutoff]
    growth = growth.drop("Australia")

    # plot growth rates
    _fig, ax = plt.subplots()
    ax = growth.sort_values().plot(kind="barh", ax=ax)
    ax.tick_params(axis="both", which="major", labelsize="x-small")
    finalise_plot(
        ax,
        title=f"{TITLE} in the 12 months\nto {combined.index[-1]} by Sector over {base_start}-{str(base_end)[2:]} Ave",
        xlabel="Growth (%)",
        rfooter=SOURCE,
        rheader=f"Industry sectors with more than {cutoff}/year insolvencies in the {base_start}-{base_end} period.",
        lheader="Last month of (provisional) data excluded",
        lfooter=LFOOTER,
        show=SHOW,
    )


plot_industrial()

## Finished

In [14]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Thu Apr 04 2024 11:30:49

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

pandas    : 2.2.1
matplotlib: 3.8.3

Watermark: 2.4.3



In [15]:
print("done.")

done.
